In [ ]:
from dag_perf_utility import *
import pandas as pd
import numpy as np
import seaborn as sn

This estimation of the performance include:

- Dynamic allocation
- Waiting time between the queue of jobs
- Estimation of the minimization, boresch restraint estimation and gmx_MMPBSA steps

In [ ]:

# thrombin, hardware 3
# This performance is on:
# AMD Ryzen Threadripper PRO 3975WX 32-Cores: 10 / NVIDIA RTX A4000: 1
performance_ligand = 806  # ns/day with a dt = 0.004 ps
performance_complex = 270  # ns/day with a dt = 0.004 ps
suffix = "_thrombin_amd-ryzen_nvidia-rtxa4000"
SCALE = False
complex_type = 'soluble'



# I have to scale down the performance based on:
if SCALE:
    # https://technical.city/en/video/Tesla-T4-vs-RTX-A4000
    performance_RTX_A4000 = 48.31
    performance_Tesla_T4 = 26.69
    scaling_factor = performance_Tesla_T4 / performance_RTX_A4000

    performance_ligand *=scaling_factor
    performance_complex *=scaling_factor




# The correction factor is to estimate the computation time properly taking into account the
# integration time step. Calcualted performance are for 0.004 ps
# For a step with 0.002 ps, it will take double of that time.


# Lets assume that the steps on min and boresch can be approximated as
# a production step that takes 0.1, 0.5 and 1 ns
extra_steps = {
    "min_ligand": 0.1 * 0.004,
    "min_complex":  0.5 * 0.004,
    "boresch":  1 * 0.004,
    "gmx_mmpbsa":  1 * 0.004,
}

simulation_time = {  # ns
    "fep": {
        "ligand": {
            "equi": [extra_steps['min_ligand'], 1 * (0.004/0.002), 1.05 * (0.004/0.003), 1, 5],
            "perturbation": [[extra_steps['min_ligand'], 0.01 * (0.004/0.002), 0.1, 0.5, 10]] * 23
        },
        "complex": {
            "soluble": {
                "equi": [extra_steps['min_complex'], 1 * (0.004/0.002), 1.05 * (0.004/0.003), 1.00005 * (0.004/0.003), 5, 10, extra_steps['boresch']],
                "perturbation": [[extra_steps['min_complex'], 0.01 * (0.004/0.003), 0.1, 0.5, 10]] * 44
            },
            "membrane": {
                "equi": [extra_steps['min_complex'], 0.125 * (0.004/0.001), 0.125 * (0.004/0.001), 0.125 * (0.004/0.001), 0.5 * (0.004/0.002), 0.5 * (0.004/0.002), 0.5 * (0.004/0.002), 10, extra_steps['boresch']],
                "perturbation": [[extra_steps['min_complex'], 0.01 * (0.004/0.002), 0.1, 0.5, 10]] * 44
            },
        }
    },
    "mmgbsa": {
        "complex": {
            "soluble": {
                "equi": [extra_steps['min_complex'], 10/1000 * (0.004/0.002), 15/1000 * (0.004/0.003), 22.5/1000 * (0.004/0.003), 60/1000, 950/1000, extra_steps['gmx_mmpbsa']], # Here I had the time in ps, the factor 1000 is to convert to ns
            },
            "membrane": {
                "equi": [extra_steps['min_complex'], 5/1000 * (0.004/0.001), 5/1000 * (0.004/0.001), 5/1000 * (0.004/0.001), 15/1000 * (0.004/0.002), 15/1000 * (0.004/0.002), 45/1000 * (0.004/0.002), 950/1000, extra_steps['gmx_mmpbsa']],
            },
        },
        "samples": [100/1000] * 20
    }
}

In [ ]:
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

EXECUTE = False

# Parameters
num_ligands = range(10, 1010, 10)
num_computers = range(10, 210, 10)

# FEP

In [ ]:
if EXECUTE:
    # Initialize results array
    makespan_data = np.zeros((len(num_ligands), len(num_computers)))

    # Initialize tqdm progress bar
    progress_bar = tqdm(total=len(num_ligands) * len(num_computers), desc="Computing Makespan", ncols=100)

    # List to hold futures
    futures = []

    # Function to update the progress bar
    def update_progress(future):
        progress_bar.update(1)

    # Parallelize using ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        for _, ligs in enumerate(num_ligands):
            for _, comps in enumerate(num_computers):
                future = executor.submit(
                    compute_makespan_fep,
                    ligs, comps,
                    performance_ligand,
                    performance_complex,
                    simulation_time["fep"]["ligand"]["equi"],
                    simulation_time["fep"]["ligand"]["perturbation"],
                    simulation_time["fep"]["complex"][complex_type]["equi"],
                    simulation_time["fep"]["complex"][complex_type]["perturbation"],
                    configure_node_wait_queue_time=60/3600)
                future.add_done_callback(update_progress)
                futures.append(future)
        
        # Wait for all futures to finish
        for future in as_completed(futures):
            ligs, comps, makespan = future.result()
            # Map results to the makespan_data array
            i = num_ligands.index(ligs)
            j = num_computers.index(comps)
            makespan_data[i, j] = makespan

    # Finalize the progress bar
    progress_bar.close()
    df = pd.DataFrame(makespan_data, columns=num_computers, index=num_ligands)
    # Save the results using joblib (you can save any object here)
    pd.to_pickle(df, f'makespan_data_fep{suffix}.dfpkl')
    print(f"Results saved to 'makespan_data_fep{suffix}.dfpkl'")

# MMGBSA

In [ ]:
if EXECUTE:
    # Initialize results array
    makespan_data = np.zeros((len(num_ligands), len(num_computers)))

    # Initialize tqdm progress bar
    progress_bar = tqdm(total=len(num_ligands) * len(num_computers), desc="Computing Makespan", ncols=100)

    # List to hold futures
    futures = []

    # Function to update the progress bar
    def update_progress(future):
        progress_bar.update(1)

    # Parallelize using ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        for i, ligs in enumerate(num_ligands):
            for j, comps in enumerate(num_computers):
                future = executor.submit(
                    compute_makespan_mmgbsa,
                    ligs, comps,
                    performance_complex,
                    simulation_time["mmgbsa"]["complex"]["soluble"]["equi"],
                    simulation_time["mmgbsa"]["samples"],
                    configure_node_wait_queue_time=60/3600)
                future.add_done_callback(update_progress)
                futures.append(future)
        
        # Wait for all futures to finish
        for future in as_completed(futures):
            ligs, comps, makespan = future.result()
            # Map results to the makespan_data array
            i = num_ligands.index(ligs)
            j = num_computers.index(comps)
            makespan_data[i, j] = makespan

    # Finalize the progress bar
    progress_bar.close()

    # Save the results using joblib (you can save any object here)
    df = pd.DataFrame(makespan_data, columns=num_computers, index=num_ligands)
    pd.to_pickle(df, f'makespan_data_mmgbsa{suffix}.dfpkl')
    print(f"Results saved to 'makespan_data_mmgbsa{suffix}.dfpkl'")

# If the cost is estimated for the use across all GPUs (like running in series in one GPU)

In [ ]:
# FEP
# https://pages.awscloud.com/rs/112-TZM-766/images/GROMACS-AWS.pdf   slide 32
# g4dn.8xlarge/Tesla T4
# price = 2.18 # $/hour
price = 1 # $/hour

fep_ligand_price = ((24 * price) / performance_ligand) * (sum(simulation_time['fep']['ligand']['equi']) + sum([sum(item) for item in simulation_time['fep']['ligand']['perturbation']]))
fep_complex_soluble_price = ((24 * price) / performance_complex) * (sum(simulation_time['fep']['complex']['soluble']['equi']) + sum([sum(item) for item in simulation_time['fep']['complex']['soluble']['perturbation']]))
fep_complex_membrane_price = ((24 * price) / performance_complex) * (sum(simulation_time['fep']['complex']['membrane']['equi']) + sum([sum(item) for item in simulation_time['fep']['complex']['membrane']['perturbation']]))


mmgbsa_complex_soluble_price = ((24 * price) / performance_complex) * (sum(simulation_time['mmgbsa']['complex']['soluble']['equi']) + sum(simulation_time['mmgbsa']['samples']))
mmgbsa_complex_membrane_price = ((24 * price) / performance_complex) * (sum(simulation_time['mmgbsa']['complex']['membrane']['equi']) + sum(simulation_time['mmgbsa']['samples']))

print(f"FEP cost ($):\n\tligand = {fep_ligand_price:.2f}\n\tsoluble complex = {fep_complex_soluble_price:.2f}\n\tmembrane complex = {fep_complex_membrane_price:.2f}\n\tTotal (soluble) = {fep_ligand_price + fep_complex_soluble_price:.2f}\n\tTotal (membrane) = {fep_ligand_price + fep_complex_membrane_price:.2f}")
print(f"MMGBSA cost ($):\n\tTotal (soluble) = {mmgbsa_complex_soluble_price:.2f}\n\tTotal (membrane) = {mmgbsa_complex_membrane_price:.2f}")
